In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [2]:
jumlah_index = 50
threads_link = []
links = []
results = []
threads = []
keywords='anies'

In [3]:
def scrape_links(page_number,keywords):
    url = f"https://www.cnbcindonesia.com/search?query={keywords}&p={page_number}&kanal=&tipe=&date="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 12 links from page 31
Scraped 12 links from page 9
Scraped 12 links from page 8
Scraped 12 links from page 10
Scraped 12 links from page 38
Scraped 12 links from page 2
Scraped 12 links from page 1
Scraped 12 links from page 27
Scraped 12 links from page 15
Scraped 12 links from page 18
Scraped 12 links from page 26
Scraped 12 links from page 50
Scraped 12 links from page 28
Scraped 12 links from page 12
Scraped 12 links from page 25
Scraped 12 links from page 29
Scraped 12 links from page 23
Scraped 12 links from page 37
Scraped 12 links from page 24
Scraped 12 links from page 46
Scraped 12 links from page 36
Scraped 12 links from page 13
Scraped 12 links from page 6
Scraped 12 links from page 45
Scraped 12 links from page 30
Scraped 12 links from page 35
Scraped 12 links from page 33
Scraped 12 links from page 40
Scraped 12 links from page 41
Scraped 12 links from page 21
Scraped 12 links from page 44
Scraped 12 links from page 19
Scraped 12 links from page 43
Scraped 12 link

In [5]:
results = []
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[1].strip()
                author_text = author_text.split(',')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('ul', {"class": "breadcrumb"})
            if category_elements:
                category_text = category_elements.find_all('li')
                category_text= category_text[2].get_text()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail_text"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            results.append({'title': title_text,
                            'keywords': keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 600


,title,keywords,author,category,date,content,link
0,Airlangga Soal Golkar tidak akan Dukung Anies:...,anies,Emir Yanwardhana,Berita,03 August 2023 12:45,"Jakarta, CNBC Indonesia - Ketua Umum Partai Go...",https://www.cnbcindonesia.com/news/20230803111...
1,Diam-Diam Piala Dunia U-17 Mau Dibuka di JIS W...,anies,Emir Yanwardhana,Berita,02 August 2023 15:11,"Jakarta, CNBC Indonesia - Menteri Pemuda dan O...",https://www.cnbcindonesia.com/news/20230802150...
2,Kandidat Cawapres Anies Diajak Gabung Timses G...,anies,Wiji Nur Hayat,Berita,02 August 2023 12:16,"Jakarta, CNBC Indonesia - Salah satu kandidat ...",https://www.cnbcindonesia.com/news/20230802121...
3,Potret Anies Berbatik Hijau Datang ke Markas P...,anies,Redaksi,Foto News,11 September 2023 18:10,Content not found,https://www.cnbcindonesia.com/news/20230911175...
4,JIS Dipakai Piala Dunia U-17 Hingga Media Asin...,anies,cnbc indonesia,Video News,02 August 2023 20:30,"Jakarta, CNBC Indonesia - Mantan Gubernur DKI ...",https://www.cnbcindonesia.com/news/20230802182...
5,Ini Bocoran Pembicaraan Penting Anies-Cak Imin...,anies,Wiji Nur Hayat,Berita,11 September 2023 18:32,"Jakarta, CNBC Indonesia - Bakal Calon Presiden...",https://www.cnbcindonesia.com/news/20230911182...
6,Anies Muncul! Buka Suara Soal Sodetan Ciliwung...,anies,Wiji Nur Hayat,Berita,03 August 2023 06:20,"Jakarta, CNBC Indonesia - Bakal Calon Presiden...",https://www.cnbcindonesia.com/news/20230802200...
7,"FIFA Umumkan Keputusan Soal JIS Warisan Anies,...",anies,Wiji Nur Hayat,Berita,03 August 2023 14:36,"Jakarta, CNBC Indonesia - FIFA sudah mengeluar...",https://www.cnbcindonesia.com/news/20230803142...
8,"UI Kalah! UGM Bisa Kirim 2 Capres, 3 Cawapres:...",anies,Riset,Berita Research,11 September 2023 07:15,"Jakarta, CNBC Indonesia - Mayoritas bakal calo...",https://www.cnbcindonesia.com/research/2023091...
9,Ini Peta Kekuatan PDI-P! Anies- Prabowo Mesti ...,anies,Tim Riset,Berita Research,12 September 2023 07:25,"Jakarta, CNBC Indonesia - Partai Demokrasi Ind...",https://www.cnbcindonesia.com/research/2023091...


In [11]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 564


,title,keywords,author,category,date,content,link
0,Airlangga Soal Golkar tidak akan Dukung Anies:...,anies,Emir Yanwardhana,Berita,03 August 2023 12:45,"Jakarta, CNBC Indonesia - Ketua Umum Partai Go...",https://www.cnbcindonesia.com/news/20230803111...
1,Diam-Diam Piala Dunia U-17 Mau Dibuka di JIS W...,anies,Emir Yanwardhana,Berita,02 August 2023 15:11,"Jakarta, CNBC Indonesia - Menteri Pemuda dan O...",https://www.cnbcindonesia.com/news/20230802150...
2,Kandidat Cawapres Anies Diajak Gabung Timses G...,anies,Wiji Nur Hayat,Berita,02 August 2023 12:16,"Jakarta, CNBC Indonesia - Salah satu kandidat ...",https://www.cnbcindonesia.com/news/20230802121...
3,Potret Anies Berbatik Hijau Datang ke Markas P...,anies,Redaksi,Foto News,11 September 2023 18:10,Content not found,https://www.cnbcindonesia.com/news/20230911175...
4,JIS Dipakai Piala Dunia U-17 Hingga Media Asin...,anies,cnbc indonesia,Video News,02 August 2023 20:30,"Jakarta, CNBC Indonesia - Mantan Gubernur DKI ...",https://www.cnbcindonesia.com/news/20230802182...
5,Ini Bocoran Pembicaraan Penting Anies-Cak Imin...,anies,Wiji Nur Hayat,Berita,11 September 2023 18:32,"Jakarta, CNBC Indonesia - Bakal Calon Presiden...",https://www.cnbcindonesia.com/news/20230911182...
6,Anies Muncul! Buka Suara Soal Sodetan Ciliwung...,anies,Wiji Nur Hayat,Berita,03 August 2023 06:20,"Jakarta, CNBC Indonesia - Bakal Calon Presiden...",https://www.cnbcindonesia.com/news/20230802200...
7,"FIFA Umumkan Keputusan Soal JIS Warisan Anies,...",anies,Wiji Nur Hayat,Berita,03 August 2023 14:36,"Jakarta, CNBC Indonesia - FIFA sudah mengeluar...",https://www.cnbcindonesia.com/news/20230803142...
8,"UI Kalah! UGM Bisa Kirim 2 Capres, 3 Cawapres:...",anies,Riset,Berita Research,11 September 2023 07:15,"Jakarta, CNBC Indonesia - Mayoritas bakal calo...",https://www.cnbcindonesia.com/research/2023091...
9,Ini Peta Kekuatan PDI-P! Anies- Prabowo Mesti ...,anies,Tim Riset,Berita Research,12 September 2023 07:25,"Jakarta, CNBC Indonesia - Partai Demokrasi Ind...",https://www.cnbcindonesia.com/research/2023091...


Save File

In [9]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/cnbc_{keywords}_{current_datetime}.xlsx'
filtered_df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')